In [1]:
import pickle
import pandas as pd

enron_df = pd.read_pickle('enron_students.pkl')

# # Read dictionary pkl file
# with open('email_to_departments.pkl', 'rb') as fp:
#     email_to_departments = pickle.load(fp)
#     print('email_to_departments dictionary loaded from pkl file')

In [2]:
# data cleaning
enron_df['To'] = enron_df['To'].fillna('')
enron_df['From'] = enron_df['From'].fillna('')
enron_df['X-From'] = enron_df['X-From'].fillna('')
enron_df['X-To'] = enron_df['X-To'].fillna('')
enron_df['X-cc'] = enron_df['X-cc'].fillna('')
enron_df['X-bcc'] = enron_df['X-bcc'].fillna('')


In [3]:
import re
import pandas as pd

def extract_emails(text):
    if pd.isna(text):
        return []
    # Regex to match email addresses
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    return re.findall(email_pattern, text)

def extract_single_email_from_text(text):
    emails = extract_emails(text)
    if (len(emails) > 1):
        print(f"Multiple emails found in: {text}")
    elif len(emails) == 1:
        return emails[0]
    print(f"Could not extract email from: {text}")
    return None

def extract_sent_to_emails(text):
    emails = extract_emails(text)
    if len(emails) > 0:
        return emails
    return []

def is_between_ect_and_ees(email_to_depart_dict, row, is_including_cc_bcc=False, additional_check=True):
    sender_email = extract_single_email_from_text(row['From'])
    if sender_email in email_to_depart_dict:
        sender_department = email_to_depart_dict[sender_email]
        if sender_department != 'ECT' and sender_department != 'EES':
            return False
        else:
            if is_including_cc_bcc:
                reciever_emails = extract_sent_to_emails(row['To']) + extract_sent_to_emails(row['X-To'])
            else:
                reciever_emails = extract_sent_to_emails(row['To']) + extract_sent_to_emails(row['X-To']) + extract_sent_to_emails(row['X-cc']) + extract_sent_to_emails(row['X-bcc'])
            
            for email in reciever_emails:
                if email in email_to_depart_dict:
                    reciever_department = email_to_depart_dict[email]

                    if sender_department == 'ECT' and reciever_department == 'EES':
                        return True
                    elif sender_department == 'EES' and reciever_department == 'ECT':
                        return True
                    
            if additional_check:
                if 'ECT' in row['X-To']:
                    if 'EES' in row['X-From']:
                        return True
                if 'EES' in row['X-To']:
                    if 'ECT' in row['X-From']:
                        return True
      
            return False
    
    else:
        if additional_check:
            if 'ECT' in row['X-To']:
                if 'EES' in row['X-From']:
                    return True
            if 'EES' in row['X-To']:
                if 'ECT' in row['X-From']:
                    return True
        return False
    


    

In [4]:

def build_dict():
    email_to_departments = {}
    email_to_departments.update({
            'smith.day@enron.com' : 'ECT'
            ,'janet.wallis@enron.com' : 'ECT'
            ,'daren.farmer@enron.com' : 'ECT'
            ,'stuart.zisman@enron.com' : 'ECT'
            ,'melissa.murphy@enron.com' : 'ECT'
            ,'thresa.allen@enron.com' : 'ECT'
            ,'leslie.reeves@enron.com' : 'ECT'
            ,'stacey.white@enron.com' : 'ECT'
            ,'janet.wallis@enron.com' : 'ECT'
            ,'daren.farmer@enron.com' : 'ECT'
            ,'brad.nebergall@enron.com': 'ECT'
            ,'colleen.koenig@enron.com': 'EES'
            ,'john.kinser@enron.com': 'ECT'
            ,'jeff.merola@enron.com': 'EES'
            })
    
    email_to_departments.update({'eric.bass@enron.com': 'ECT'})

    multiple_count = 0
    # TODO: implement check for multiple emails in X-From and From. should be only one email
    for index, row in enron_df.iterrows():
        x_emails = extract_emails(row['X-From'])
        from_emails = set([mail.lower() for mail in (extract_emails(row['From']) + x_emails)])
        to_emails = set([mail.lower() for mail in (extract_emails(row['To']) + extract_emails(row['X-To']))])
        if len(from_emails) == 0:
            continue
        if len(from_emails) > 1:
            from_emails = [mail for mail in from_emails if not mail.startswith('imceanotes')]
            if len(from_emails) > 1 or len(from_emails) == 0:
                multiple_count += 1

        for email in from_emails:
            email = email.lower()
            # TODO: validate company email starts with enron.com
            if "enron.com" not in email and email not in email_to_departments:
                email_to_departments.update({email: 'NA'})
            else:
                if email in email_to_departments and email_to_departments[email] != 'Other':
                    continue
                elif 'EES' in row['X-From']:
                    email_to_departments.update({email: 'EES'})
                    if len(to_emails) == 1:
                        email_to_departments.update({list(to_emails)[0]: 'ECT'})
                elif 'ECT' in row['X-From']:
                    email_to_departments.update({email: 'ECT'})
                    if len(to_emails) == 1:
                        email_to_departments.update({list(to_emails)[0]: 'EES'})
                else:
                    email_to_departments.update({email: 'Other'})

        # new addition - need to check
        if len(to_emails) == 1:
            if 'ECT' in row['X-To']:
                email_to_departments.update({list(to_emails)[0]: 'ECT'})
            elif 'EES' in row['X-To']:
                email_to_departments.update({list(to_emails)[0]: 'EES'})

    email_to_departments.update({
            'smith.day@enron.com' : 'ECT'
            ,'janet.wallis@enron.com' : 'ECT'
            ,'daren.farmer@enron.com' : 'ECT'
            ,'stuart.zisman@enron.com' : 'ECT'
            ,'melissa.murphy@enron.com' : 'ECT'
            ,'thresa.allen@enron.com' : 'ECT'
            ,'leslie.reeves@enron.com' : 'ECT'
            ,'stacey.white@enron.com' : 'ECT'
            ,'janet.wallis@enron.com' : 'ECT'
            ,'daren.farmer@enron.com' : 'ECT'
            ,'brad.nebergall@enron.com': 'ECT'
            ,'colleen.koenig@enron.com': 'EES'
            ,'john.kinser@enron.com': 'ECT'
            ,'jeff.merola@enron.com	': 'EES'
            ,'scott.mills@enron.com': 'ECT'
            ,'marilyn.colbert@enron.com': 'ECT'
            ,'molly.harris@enron.com': 'ECT'
            ,'joseph.wagner@enron.com': 'ECT'
            ,'stuart.zisman@enron.com,' : 'ECT'
            ,'janet.wallis@enron.com' : 'ECT'
            ,'daren.farmer@enron.com' : 'ECT'
            ,'stephanie.gardner@enron.com' : 'EES'

            })
    
    return email_to_departments


email_to_department_dict = build_dict()

In [5]:
len(email_to_department_dict)

20638

In [6]:
enron_df['is_between_ect_and_ees'] = enron_df.apply(lambda row: is_between_ect_and_ees(email_to_department_dict, row), axis=1)

Could not extract email from: u@d.h
Could not extract email from: pep <performance.>


In [7]:
# remove all spaces 
enron_df['violated_rules'] = enron_df['violated_rules'].apply(lambda x: x.replace(' ', ''))

# apply split ',' on violated rules
enron_df['violated_rules'] = enron_df['violated_rules'].apply(lambda x: x.split(',')) 

In [8]:
def update_dict(enron_df_not_catched_1_1, email_to_depart):
    # iterate over the rows of the dataframe enron_df_not_catched_1_1
    updated = 0
    for index, row in enron_df_not_catched_1_1.iterrows():
        x_emails = extract_emails(row['X-From'])
        emails = set([mail.lower() for mail in (extract_emails(row['From']) + x_emails)])
        sent_to_emails = extract_sent_to_emails(row['To']) + extract_sent_to_emails(row['X-To'])
        if len(emails) == 0:
            continue
        if len(emails) > 1:
            emails = [mail for mail in emails if not mail.startswith('imceanotes')]
            if len(emails) > 1 or len(emails) == 0:
                # print(f"From: {row['From']} X-From: {row['X-From']} has multiple emails: {emails}")
                continue
        for email in emails:
            email = email.lower()
            # if from email is in dictionary, get the department
            if email in email_to_depart:
                depart = email_to_depart[email]
                if depart == 'ECT':
                    if len(sent_to_emails) == 1:
                        email_to_depart[sent_to_emails[0]] = 'EES'
                        updated += 1
                elif depart == 'EES':
                    if len(sent_to_emails) == 1:
                        email_to_depart[sent_to_emails[0]] = 'ECT'
                        updated += 1

    print(f"Updated {updated} emails")
    


In [9]:
enron_df_not_catched_1_1 = enron_df[enron_df['is_between_ect_and_ees'] == False & enron_df['violated_rules'].apply(lambda x: '1.1' in x)]

update_dict(enron_df_not_catched_1_1, email_to_department_dict)

Updated 198084 emails


* HERE WE START USER TO LOCATION *


In [10]:
def get_email_to_location_dict(df):
    email_to_location = {}
    for index, row in enron_df.iterrows():
        x_emails = extract_emails(row['X-From'])
        from_emails = set([mail.lower() for mail in (extract_emails(row['From']) + x_emails)])
        if '/OU=EU/' in row['X-From']:
            for email in from_emails:
                email_to_location.update({email: 'EU'})
        elif '/HOU/' in row['X-From']:
            for email in from_emails:
                email_to_location.update({email: 'NA'})
        elif '/OU=NA/' in row['X-From']:
            for email in from_emails:
                email_to_location.update({email: 'NA'})

    # iterate over the dataframe to get 
    violated_rule_1_2 = df[df['violated_rules'].apply(lambda x: '1.2' in x)]

    for index, row in violated_rule_1_2.iterrows():
        from_emails = extract_emails(row['X-From'])
        from_emails = list(set([mail.lower() for mail in (extract_emails(row['From']) + from_emails)]))
        if '/OU=EU/' in row['X-From']:
            for email in from_emails:
                email_to_location.update({email: 'EU'})
        elif '/HOU/' in row['X-From']:
            for email in from_emails:
                email_to_location.update({email: 'NA'})
        elif '/OU=NA/' in row['X-From']:
            for email in from_emails:
                email_to_location.update({email: 'NA'})
        
        sent_to_email = extract_emails(row['X-To'])
        if len(sent_to_email) == 1:
            if from_emails[0] not in email_to_location:
                continue
            if email_to_location[from_emails[0]] == 'EU':
                email_to_location.update({sent_to_email[0]: 'NA'})
            elif email_to_location[from_emails[0]] == 'NA':
                email_to_location.update({sent_to_email[0]: 'EU'})
            
    return email_to_location


email_to_location = get_email_to_location_dict(enron_df)

In [11]:
def is_between_EU_and_NA(row, email_to_location):
    from_email = extract_emails(row['From'])
    from_email = extract_emails(row['X-From']) + from_email
    from_email = list(set([mail.lower() for mail in from_email]))
    to_email = extract_emails(row['To'])
    to_email = extract_emails(row['X-To']) + to_email
    to_email = list(set([mail.lower() for mail in to_email]))
    if len(from_email) == 0 or len(to_email) == 0:
        return False
    if from_email[0] in email_to_location:
        if email_to_location[from_email[0]] == 'EU':
            for email in to_email:
                if email in email_to_location and email_to_location[email] == 'NA':
                    return True
        if email_to_location[from_email[0]] == 'NA':
            for email in to_email:
                if email in email_to_location and email_to_location[email] == 'EU':
                    return True
    
    return False

In [12]:
enron_df['is_EU_To_NA'] = enron_df.apply(lambda row: is_between_EU_and_NA(row, email_to_location), axis=1)

In [13]:
def get_c_suit_dict():
    emails = set({
        'kenneth.lay@enron.com',
        'ken.skilling@enron.com',
        'rbowers@nyiso.com',
        'michael.brown@enron.com',
        'john.sherriff@enron.com',
        'jeffrey.sherrick@enron.com',
        'a..shankman@enron.com',
        'ken.rice@enron.com',
        'greg.piper@enron.com',
        'mark.metts@enron.com',
        'coo.jeff@enron.com',
        'rebecca.mcdonald@enron.com',
        'danny.mccarty@enron.com',
        'dan.leff@enron.com',
        'john.lavorato@enron.com',
        'mark.koenig@enron.com',
        'louise.kitchen@enron.com',
        'stanley.horton@enron.com',
        '40enron@enron.com', # for some reason is tagged for stanley horton
        'ben.glisan@enron.com',
        'mark.frevert@enron.com',
        'andrew.fastow@enron.com',
        'jr..legal@enron.com',
        'derrick@enron.com',
        'david.delainey@enron.com',
        'richard.causey@enron.com',
        'michael.brown@enron.com',
        'raymond.bowen@enron.com',
    })

    violated_rule_1_3 = enron_df[enron_df['violated_rules'].apply(lambda x: '2.2' in x)]

    for index, row in violated_rule_1_3.iterrows():
        from_emails = extract_emails(row['X-From'])
        from_emails = list(set([mail.lower() for mail in (extract_emails(row['From']) + from_emails)]))
        to_email = extract_emails(row['To'])
        to_email = extract_emails(row['X-To']) + to_email
        to_email = list(set([mail.lower() for mail in to_email]))

        if len(from_emails) == 0:
            continue
        for email in from_emails:
            emails.add(email)
            # print(f"Email: {email} is in C-Suit")

        # for email in to_email:
        #     emails.add(email)
        #     print(f"Email: {email} is in C-Suit")

    return emails

c_suit_emails = get_c_suit_dict()

In [14]:
def is_in_c_suit(row, c_suit_emails):
    # Extract and clean email addresses from 'From' fields
    from_emails = extract_emails(row['X-From']) + extract_emails(row['From'])
    from_emails = list(set(email.lower() for email in from_emails))

    # Extract and clean email addresses from 'To' fields
    to_email = extract_emails(row['X-To']) + extract_emails(row['To'])
    to_email = list(set(email.lower() for email in to_email))

    # Check if both sender and recipient are in the c-suite email list
    return any(email in c_suit_emails for email in from_emails) and any(email in c_suit_emails for email in to_email)



In [15]:
enron_df['is_c_suit'] = enron_df.apply(lambda row: is_in_c_suit(row, c_suit_emails), axis=1)

In [16]:
def get_from_to_emails(row):
    from_emails = extract_emails(row['X-From']) + extract_emails(row['From'])
    from_emails = list(set(email.lower() for email in from_emails))

    # Extract and clean email addresses from 'To' fields
    to_email = extract_emails(row['X-To']) + extract_emails(row['To'])
    to_email = list(set(email.lower() for email in to_email))

    return from_emails, to_email

In [17]:
def is_leaving_corporate(row):
    from_emails, to_email = get_from_to_emails(row)
    if len(from_emails) == 0 or len(to_email) == 0:
        return False
    
    for email in to_email:
        if 'enron.com' not in email:
            return True

    return False


def is_leaving_ect(row, email_to_depart):
    from_emails, to_email = get_from_to_emails(row)
    if len(from_emails) == 0 or len(to_email) == 0:
        return False

    for email in from_emails:
        if email in email_to_depart and email_to_depart[email] == 'ECT':
            for t_email in to_email:
                if t_email in email_to_depart and email_to_depart[t_email] != 'ECT':
                    return True

    return False

In [18]:
def is_violating_rule_1_1(row):
    return is_between_ect_and_ees(email_to_department_dict, row, is_including_cc_bcc=False, additional_check=True)

def is_violating_rule_1_2(row):
    return is_between_EU_and_NA(row, email_to_location)

def is_violating_rule_1_3(row):
    return is_leaving_corporate(row)

def is_violating_rule_2_1(row):
    return is_leaving_ect(row, email_to_department_dict)

def is_violating_rule_2_2(row):
    return is_in_c_suit(row, c_suit_emails)

def is_violating_rule_2_3(row):
    return is_leaving_corporate(row, email_to_department_dict)

In [19]:
def classify_mail (mail_content: str) -> bool:
    pass
    

In [7]:
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore

def build_lda_model(df, num_topics=3, threshold=0.3):
    # Adjust number of topics for "None of these topics" category
    num_topics += 1

    # Extract and tokenize the email body
    df['tokens'] = df['email_text'].apply(lambda x: x.split())

    # Create a dictionary representation of the documents
    dictionary = corpora.Dictionary(df['tokens'])
    print("Number of unique tokens before filtering:", len(dictionary))
    dictionary.filter_extremes(no_below=5, no_above=0.5)
    print("Number of unique tokens after filtering:", len(dictionary))

    # Create bag-of-words representation of the documents
    df['bow'] = df['tokens'].apply(dictionary.doc2bow)
    # Filter out documents that result in no words after processing
    valid_df = df[df['bow'].map(len) > 0]

    # Check if there are any valid documents left
    if valid_df.empty:
        raise ValueError("No valid documents to process. All documents resulted in empty BOW representations.")

    # Build an LDA model
    lda_model = LdaMulticore(corpus=list(valid_df['bow']),
                             id2word=dictionary,
                             num_topics=num_topics,
                             chunksize=100,
                             passes=10,
                             per_word_topics=True,
                             workers=11)

    return lda_model, valid_df, dictionary


def get_topic_assignments(lda_model: LdaMulticore, bow_corpus, threshold=0.3):
    
    # Assign documents to topics
    topic_assignments = []
    for bow in bow_corpus:
        topic_probs = lda_model.get_document_topics(bow, minimum_probability=0)
        # Sort topics by probability
        topic_probs = sorted(topic_probs, key=lambda x: x[1], reverse=True)
        # Assign "None of these topics" if the highest probability is below the threshold
        if topic_probs[0][1] < threshold:
            topic_assignments.append("None of these topics")
        else:
            topic_assignments.append(f"Topic {topic_probs[0][0]}")

    return topic_assignments


In [22]:
df_labeled1_1 = enron_df[(enron_df['violated_rules'].apply(lambda x: '1.1' in x))]
df_labeled1_2 = enron_df[(enron_df['violated_rules'].apply(lambda x: '1.2' in x))]
df_labeled1_3 = enron_df[(enron_df['violated_rules'].apply(lambda x: '1.3' in x))]

enron_df['is_violating_rule_1_1'] = enron_df.apply(is_violating_rule_1_1, axis=1)
enron_df['is_violating_rule_1_2'] = enron_df.apply(is_violating_rule_1_2, axis=1)
enron_df['is_violating_rule_1_3'] = enron_df.apply(is_violating_rule_1_3, axis=1)

df_not_label_11 = enron_df[enron_df['is_violating_rule_1_1'] == True & enron_df['violated_rules'].apply(lambda x: '1.1' not in x)]
df_not_label_12 = enron_df[enron_df['is_violating_rule_1_2'] == True & enron_df['violated_rules'].apply(lambda x: '1.2' not in x)]
df_not_label_13 = enron_df[enron_df['is_violating_rule_1_3'] == True & enron_df['violated_rules'].apply(lambda x: '1.3' not in x)]



Could not extract email from: u@d.h
Could not extract email from: pep <performance.>


In [3]:
enron_df['email_text'] = enron_df['Subject'] + ' ' + enron_df['email_body']

enron_df.iloc[0]['email_text']
import preprocessing

print(preprocessing.tokenize_and_lemmatize(enron_df.iloc[0]['email_text']))

[nltk_data] Downloading package stopwords to C:\Users\Efraim
[nltk_data]     Yosofov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Efraim
[nltk_data]     Yosofov\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Here forecast


In [4]:
import preprocessing

preprocessing.data_cleaning(enron_df)

removed attachments
parsed contacts
removed foward text
filled nan values


d:\private\Cyber and Machine learning\Assignments\cyber-ai\preprocessing.py:128: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  enron_df['email_text'].fillna(' ', inplace=True)


removed email headers
removed small words
lemmatized


In [5]:
enron_df.head(10)

,Date,From,To,X-To,X-From,X-cc,X-bcc,Subject,email_body,verdict,violated_rules,email_text
0,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,Tim Belden <Tim Belden/Enron@EnronXGate>,Phillip K Allen,,,,Here is our forecast\n\n,ALLOW,,ere forecast
1,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,John J Lavorato <John J Lavorato/ENRON@enronXg...,Phillip K Allen,,,Re:,Traveling to have a business meeting takes the...,ALLOW,,raveling business meeting take fun trip specia...
2,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",phillip.allen@enron.com,leah.arsdall@enron.com,Leah Van Arsdall,Phillip K Allen,,,Re: test,test successful. way to go!!!,ALLOW,,test test successful way
3,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,randall.gay@enron.com,Randall L Gay,Phillip K Allen,,,,"Randy,\n\n Can you send me a schedule of the s...",ALLOW,,andy send schedule salary level everyone sched...
4,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,Greg Piper,Phillip K Allen,,,Re: Hello,Let's shoot for Tuesday at 11:45.,ALLOW,,ello shoot uesday
5,"Thu, 31 Aug 2000 04:17:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,Greg Piper,Phillip K Allen,,,Re: Hello,"Greg,\n\n How about either next Tuesday or Thu...",ALLOW,,ello reg either next uesday hursday hillip
6,"Tue, 22 Aug 2000 07:44:00 -0700 (PDT)",phillip.allen@enron.com,"david.l.johnson@enron.com, john.shafer@enron.com","david.l.johnson@enron.com, John Shafer",Phillip K Allen,,,,Please cc the following distribution list with...,ALLOW,,lease following distribution list update hilli...
7,"Fri, 14 Jul 2000 06:59:00 -0700 (PDT)",phillip.allen@enron.com,joyce.teixeira@enron.com,Joyce Teixeira,Phillip K Allen,,,Re: PRC review - phone calls,any morning between 10 and 11:30,ALLOW,,review phone call morning
8,"Tue, 17 Oct 2000 02:26:00 -0700 (PDT)",phillip.allen@enron.com,mark.scott@enron.com,Mark Scott,Phillip K Allen,,,Re: High Speed Internet Access,1. login: pallen pw: ke9davis\n\n I don't thi...,BLOCK,2.3,igh peed nternet ccess login pallen davis thin...
9,"Mon, 16 Oct 2000 06:44:00 -0700 (PDT)",phillip.allen@enron.com,zimam@enron.com,zimam@enron.com,Phillip K Allen,,,FW: fixed forward or other Collar floor gas pr...,---------------------- Forwarded by Phillip K ...,BLOCK,2.1,fixed forward ollar floor gas price term fixed...


In [5]:
# save dataframe to pickle file
enron_df.to_pickle('enron_df_cleaned.pkl')

In [21]:
enron_df['email_text'] = enron_df['email_text'].apply(lambda x: ' '.join(x))

KeyboardInterrupt: 

In [20]:
enron_df.head()

# import preprocessing
# print(preprocessing.remove_foward_text('asdf ekl asdf ;e,fasdf efadsf '))

,Date,From,To,X-To,X-From,X-cc,X-bcc,Subject,email_body,verdict,violated_rules,email_text,tokens,bow
0,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,Tim Belden <Tim Belden/Enron@EnronXGate>,Phillip K Allen,,,,Here is our forecast\n\n,ALLOW,,e r e f o r e c a s t,"[e, r, e, f, o, r, e, c, a, s, t]",[]
1,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,John J Lavorato <John J Lavorato/ENRON@enronXg...,Phillip K Allen,,,Re:,Traveling to have a business meeting takes the...,ALLOW,,r a v e l i n g b u s ...,"[r, a, v, e, l, i, n, g, b, u, s, i, n, e, s, ...","[(0, 1), (1, 1)]"
2,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",phillip.allen@enron.com,leah.arsdall@enron.com,Leah Van Arsdall,Phillip K Allen,,,Re: test,test successful. way to go!!!,ALLOW,,t e s t t e s t s u ...,"[t, e, s, t, t, e, s, t, s, u, c, c, e, s, s, ...",[]
3,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,randall.gay@enron.com,Randall L Gay,Phillip K Allen,,,,"Randy,\n\n Can you send me a schedule of the s...",ALLOW,,a n d y s e n d s c ...,"[a, n, d, y, s, e, n, d, s, c, h, e, d, u, l, ...",[]
4,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,Greg Piper,Phillip K Allen,,,Re: Hello,Let's shoot for Tuesday at 11:45.,ALLOW,,e l l o s h o o t u ...,"[e, l, l, o, s, h, o, o, t, u, e, s, d, a, y]",[]


In [9]:
# build the LDA model
# lda_model, bow_corpus, dictionary = build_lda_model(combined_df, num_topics=3, threshold=0.3)
try:
    lda_model, bow_corpus, dictionary = build_lda_model(enron_df, num_topics=20, threshold=0.3)
    print("Model built successfully with {} topics.".format(lda_model.num_topics))
except ValueError as e:
    print(e)

AttributeError: 'list' object has no attribute 'split'

In [51]:
# create dictionary and save the model and dictionary

lda_model.save('lda_model.pkl')
dictionary.save('dictionary.pkl')




In [46]:
# # Assign topics to documents
# def get_most_probable_topic(doc, lda_model):
#     if not doc:
#         return None  # Handling for any empty document cases
#     sorted_topics = sorted(lda_model.get_document_topics(doc), key=lambda x: x[1], reverse=True)
#     return sorted_topics[0][0] if sorted_topics else None

# combined_df['Topic'] = [get_most_probable_topic(doc, lda_model) for doc in bow_corpus]


def get_most_probable_topic(doc, model, get_destribution=False):
    if not doc:
        return None  # Handle empty documents
    topic_probs = model.get_document_topics(doc, minimum_probability=0)
    if not topic_probs:
        return None  # Handle cases with no significant topic probability
    
    if get_destribution:
        return topic_probs
    
    return max(topic_probs, key=lambda x: x[1])[0]  # Return only the topic index

# Assume lda_model and valid_df are from your previous output
bow_corpus['Topic'] = [get_most_probable_topic(doc, lda_model, True) for doc in bow_corpus['bow']]


C:\Users\Efraim Yosofov\AppData\Local\Temp\ipykernel_29232\1861742893.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bow_corpus['Topic'] = [get_most_probable_topic(doc, lda_model, True) for doc in bow_corpus['bow']]


In [28]:
bow_corpus.dropna(subset=['Subject', 'X-cc', 'X-bcc', 'X-From', 'X-To'], inplace=True)

In [45]:
# bow_corpus[['violated_rules', 'Topic']].head(30)
pd.set_option('display.max_colwidth', None)
# bow_corpus[bow_corpus['violated_rules'].apply(lambda x: '1.1' in x)][['violated_rules', 'Topic']].head(30)

bow_corpus[['violated_rules', 'Topic']].head(30)

KeyError: "['Topic'] not in index"

In [47]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, list(bow_corpus['bow']), dictionary)
pyLDAvis.display(vis)


In [31]:
# Calculate perplexity for LDA model
perplexity = lda_model.log_perplexity(list(bow_corpus['bow']))
print("Perplexity: ", perplexity)


Perplexity:  -7.578660501773334


In [50]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=bow_corpus['tokens'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)


Coherence Score:  0.5945637667754743
